In [1]:
!pip install keras-tuner

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 2.7 MB/s eta 0:00:00


In [2]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [3]:
print(tf.__version__)

2.17.0


In [4]:
fashion_mnist = keras.datasets.fashion_mnist

In [5]:
(train_images,train_labels),(test_images,test_labels) = fashion_mnist.load_data()

29515/29515 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
26421880/26421880 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
5148/5148 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
4422102/4422102 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [6]:
#Normalization
train_images = train_images/255.0
test_images = test_images/255.0

In [7]:
train_images[0]

array([[0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.    

In [8]:
train_images.shape

(60000, 28, 28)

In [9]:
train_images = train_images.reshape(len(train_images),28,28,1)
test_images = test_images.reshape(len(test_images),28,28,1)

In [118]:
def build_model(hp):
    # create model object
    model = keras.Sequential([
    #adding first convolutional layer
    keras.layers.Conv2D(
        #adding filter
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
        # adding filter size or kernel size
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        #activation function
        activation='relu',
        input_shape=(28,28,1),
        kernel_regularizer = keras.regularizers.l2(0.0005)),
    # adding second convolutional layer
    keras.layers.Conv2D(
        #adding filter
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
        #adding filter size or kernel size
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        #activation function
        activation='relu',
        kernel_regularizer = keras.regularizers.l2(0.0003)
    ),
    # adding flatten layer
    keras.layers.Flatten(),
    # adding dense layer
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu',
        kernel_regularizer = keras.regularizers.l2(0.0003)
    ),
    # output layer
    keras.layers.Dense(10, activation='softmax')
    ])
    #compilation of model
    model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
    return model

In [119]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [120]:
tuner_search = RandomSearch(build_model,objective='val_accuracy',max_trials=5,directory='output',project_name="Mnist Fashion")

Reloading Tuner from output/Mnist Fashion/tuner0.json


In [121]:
tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.1)

In [122]:
model = tuner_search.get_best_models(num_models=1)[0]

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.10/dist-packages/keras/src/saving/saving_lib.py:576: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 18 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [123]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 24, 24, 48)          │           1,248 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 22, 22, 48)          │          20,784 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 23232)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 48)                  │       1,115,184 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 10)                  │             490 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,137,706 (4.34 MB)

 Trainable params: 1,137,706 (4.34 MB)

 Non-trainable params: 0 (0.00 B)

In [124]:
model.fit(train_images,train_labels,epochs=10,validation_split=0.1,initial_epoch=3)

Epoch 4/10
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 9s 4ms/step - accuracy: 0.9042 - loss: 0.3302 - val_accuracy: 0.8997 - val_loss: 0.3471
Epoch 5/10
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9138 - loss: 0.3135 - val_accuracy: 0.9020 - val_loss: 0.3496
Epoch 6/10
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9184 - loss: 0.3065 - val_accuracy: 0.8982 - val_loss: 0.3671
Epoch 7/10
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9189 - loss: 0.3001 - val_accuracy: 0.9045 - val_loss: 0.3573
Epoch 8/10
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 11s 3ms/step - accuracy: 0.9270 - loss: 0.2899 - val_accuracy: 0.9093 - val_loss: 0.3482
Epoch 9/10
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 10s 3ms/step - accuracy: 0.9277 - loss: 0.2838 - val_accuracy: 0.9027 - val_loss: 0.3637
Epoch 10/10
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9332 - loss: 0.2766 - val_accuracy: 0.9127 - val_loss: 0.3529


In [125]:
predictions = model.predict(test_images)

313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step


In [126]:
print(test_labels)

[9 2 1 ... 8 1 5]


In [127]:
print(predictions)

[[7.98156066e-07 5.08164444e-09 3.78354230e-07 ... 2.44033919e-03
  5.15677584e-06 9.96974349e-01]
 [2.74586910e-03 1.04955573e-08 9.89191413e-01 ... 4.89712645e-11
  8.58908464e-08 1.12570994e-07]
 [8.96111669e-06 9.99990582e-01 1.02895520e-07 ... 1.06583561e-11
  7.82080001e-09 1.49863844e-10]
 ...
 [7.80212868e-05 2.49475303e-08 3.34692362e-04 ... 1.22675416e-08
  9.99259889e-01 1.10363771e-07]
 [7.49143464e-06 9.99952316e-01 8.88509817e-08 ... 1.61774608e-10
  1.07317888e-08 9.92492044e-09]
 [2.95816251e-04 2.94665483e-07 9.76483716e-05 ... 7.87115172e-02
  6.91673011e-02 9.79259983e-03]]


In [128]:
y_pred = np.argmax(predictions, axis=1)
print(y_pred)

[9 2 1 ... 8 1 5]


In [129]:
from sklearn.metrics import accuracy_score

y_pred = np.argmax(predictions, axis=1)
accuracy_score(test_labels, y_pred)

0.9048